# 1、模型结构搭建

深度学习与经典机器学习的一个最大的区别在于模型结构方面，经典机器学习模型往往有着固定的范式和结构，例如：随机森林就是由指定数量的决策树构成，虽然这里的n_estimators可以任选，但整体来看随机森林模型的结构是确定的；而深度学习模型的基础在于神经网络，即由若干的神经网络层构成，每一层使用的神经网络模块类型可以不同（全连接层、卷积层等等），包含的神经元数量差异也会带来很大的不同。也正因如此，深度学习给使用者提供了更大的设计创新空间。

当然，网络架构（Architecture）的设计不需要从零开始，PyTorch这些深度学习框架的一大功能就是提供了基础的神经网络模块（Module），而使用者仅需根据自己的设计意图将其灵活组装起来即可——就像搭积木一般！

In [3]:
from torch import nn
import torch

## 1.1线性层（全连接层）
- 功能：将所有输入特征连接起来，用于综合特征并进行分类或回归。
- 特点：
    - 每个神经元与上一层的所有神经元相连。
    - 输出的大小由神经元的数量决定。
- 应用：通常在模型的最后几层，用于最终输出预测。

In [8]:
m = nn.Linear(20,30)
inputs = torch.randn(128, 20)
print(inputs.shape)
output = m(inputs)
print(output.size())

torch.Size([128, 20])
torch.Size([128, 30])


## 1.2卷积层
- 功能：提取局部特征，特别适用于图像和时序数据。
- 特点：
    - 使用卷积核（filter）扫描输入数据，提取边缘、纹理等局部特征。
    - 可调参数：卷积核大小、步长（stride）、填充（padding）等。
- 应用：主要用于卷积神经网络（CNN）中，如图像分类、目标检测等任务

In [ ]:
m = nn.Conv2d(in_channels=16, out_channels=33, kernel_size=3, stride=2)
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.randn(20, 16, 50, 100)
output = m(input)
print(output.shape)

torch.Size([20, 33, 24, 49])


## 1.3池化层
- 功能：降维和压缩特征，减少计算量，提高模型的鲁棒性。
- 类型：
    - 最大池化（Max Pooling）：取窗口内的最大值。
    - 平均池化（Average Pooling）：取窗口内的平均值。
- 应用：通常与卷积层交替使用。

In [6]:
m = nn.MaxPool2d(3, stride=2)
# m = nn.MaxPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m(input)
print(output.shape)

torch.Size([20, 16, 24, 31])


## 1.4激活函数
- 功能：引入非线性，增强模型的表达能力。
- 常见激活函数：
    - ReLU（Rectified Linear Unit）：常用于隐藏层。
    - Sigmoid：常用于二分类问题的输出层。
    - Softmax：用于多分类问题的输出层。
    - Tanh：用于某些特定场景。

In [8]:
import torch
import torch.nn.functional as F

x = torch.randn(1, 3)  

output_f = F.relu(x)
print("Output with F.relu:", output_f)


Output with F.relu: tensor([[1.3166, 0.2974, 0.7103]])


## 1.5归一化层
- 功能：加速训练，提高模型稳定性。
- 类型：
    - 批归一化（Batch Normalization）：对每个批次的数据进行归一化。
    - 层归一化（Layer Normalization）：对每一层的数据进行归一化。
- 应用：常用于深度网络中以防止梯度消失或梯度爆炸。

In [10]:
m = nn.BatchNorm2d(100)
input = torch.randn(20, 100, 35, 45)
output = m(input)
print(output.shape)

torch.Size([20, 100, 35, 45])


## 1.6循环层
- 功能：用于处理序列数据，捕获时间依赖性。
- 类型：
    - 简单循环网络（RNN）。
    - 长短期记忆网络（LSTM）。
    - 门控循环单元（GRU）。
- 应用：主要用于自然语言处理、时间序列预测等任务。

In [11]:
rnn = nn.RNN(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)


In [12]:
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))

In [13]:
rnn = nn.GRU(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
output, hn = rnn(input, h0)

## 1.7Dropout层
- 功能：随机丢弃部分神经元，防止过拟合。
- 特点：
    - 在训练阶段随机丢弃，推理阶段保留所有神经元。
    - 丢弃比例是一个超参数（如 0.5 表示丢弃 50%）

In [14]:
m = nn.Dropout2d(p=0.2)
input = torch.randn(20, 16, 32, 32)
output = m(input)

## 1.8完整的模型搭建

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义 MLP 模型
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        
        # 定义输入层到隐藏层的全连接层
        self.fc1 = nn.Linear(input_size, hidden_size)
        
        # 定义隐藏层到输出层的全连接层
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        # 激活函数
        self.relu = nn.ReLU()
        
        # 输出层使用 Sigmoid 激活函数（因为是二分类问题）
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 输入通过第一个全连接层并激活
        x = self.fc1(x)
        x = self.relu(x)
        
        # 输入通过第二个全连接层
        x = self.fc2(x)
        
        # 使用 Sigmoid 激活函数得到概率值
        x = self.sigmoid(x)
        
        return x

# 实例化 MLP 模型
input_size = 10  # 输入特征的维度
hidden_size = 64  # 隐藏层的维度
output_size = 1  # 输出层的维度（对于二分类问题，输出层通常只有一个神经元）

model = MLP(input_size, hidden_size, output_size)

# 打印模型结构
print(model)


MLP(
  (fc1): Linear(in_features=10, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


![alexnex](./Snipaste_2025-01-18_11-24-28.png)


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        
        # 定义卷积层
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),  # 第1层卷积
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # 最大池化
            
            nn.Conv2d(96, 256, kernel_size=5, padding=2),  # 第2层卷积
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # 最大池化
            
            nn.Conv2d(256, 384, kernel_size=3, padding=1),  # 第3层卷积
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 384, kernel_size=3, padding=1),  # 第4层卷积
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 256, kernel_size=3, padding=1),  # 第5层卷积
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)  # 最大池化
        )
        
        # 定义全连接层
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),  # 256通道，6x6图像大小
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)  # 最后一层对应类别数量
        )
    
    def forward(self, x):
        x = self.features(x)  # 通过卷积层
        x = torch.flatten(x, 1)  # 展平
        x = self.classifier(x)  # 通过全连接层
        return x

# 示例用法
model = AlexNet(num_classes=10)  # 用于10类分类
print(model)

# 打印模型概况
summary(model, input_size=(3, 224, 224))  # 输入图像大小为224x224


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, 

# 2、损失函数
- 功能：衡量模型预测值与真实值之间的差异，为优化算法提供优化方向和依据。

- 类型：

    - 回归损失函数：
        - 均方误差（MSE）：对预测值与真实值的误差平方求均值，强调较大误差的惩罚。
        - 均绝误差（MAE）：对预测值与真实值的绝对误差求均值，更鲁棒但对大误差不敏感。
        - Huber 损失：结合 MSE 和 MAE，适用于对异常值更鲁棒的场景。
    - 分类损失函数：
        - 交叉熵损失（Cross-Entropy Loss）：衡量预测分布与真实分布之间的差异，适用于二分类或多分类任务。
        - 二元交叉熵（Binary Cross-Entropy）：专用于二分类任务的交叉熵损失函数。
        - 稀疏交叉熵（Sparse Categorical Cross-Entropy）：用于多分类任务，支持整数标签形式。
    - 对比损失函数：
        - 对比损失（Contrastive Loss）：用于计算样本对的相似性，常见于嵌入学习。
        - 三元组损失（Triplet Loss）：用于优化嵌入空间，使正样本靠近锚点，负样本远离锚点。
- 应用：广泛应用于分类、回归、序列预测、嵌入学习等任务中，为模型提供衡量标准并指导参数优化。

In [17]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
output.backward()

In [18]:
# 索引
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
# 概率
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

tensor([[-2.1452, -0.6685,  0.4029, -1.4761,  2.5531],
        [ 0.1007, -0.9141,  0.8897, -0.1023,  0.6255],
        [ 0.1938, -0.1826, -0.8975, -1.2483,  0.9766]], requires_grad=True)


# 3、优化器
- 功能：优化器用于根据损失函数的梯度信息更新模型参数，从而使损失函数的值逐步减小。优化器是深度学习模型训练的关键组件，它通过迭代调整权重参数，帮助模型更快更准确地拟合数据。
- 类型：
    - SGD：简单而高效，适用于大规模数据集，但可能收敛缓慢。
    - Momentum：在梯度下降中引入动量，适用于加速收敛。
    - AdaGrad：适用于稀疏数据，自动调整每个参数的学习率。
    - RMSProp：适用于非平稳问题，特别是在训练 RNN 时表现优秀。
    - Adam：结合了动量法和 RMSProp，是最常用的优化器，适合各种任务。
    - AdamW：Adam 的改进版，适用于需要正则化的任务。
    - Nadam：结合了 Nesterov 动量的 Adam 版本，适合大多数任务。
    - Adadelta：适用于梯度爆炸或消失问题，并且无需手动调整学习率。
选择优化器时，要根据任务的具体需求、数据的特性以及模型的复杂度来决定最合适的优化算法。

In [20]:
from torch import optim
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)